**Note:** A separate notebook is created for model testing to overcome the CUDA out of memory error.

Import the libraries needed for this notebook.

In [1]:
from fastai import *
from fastai.vision import *
from fastai.callbacks.hooks import *

In [2]:
arch = models.resnet50

In [6]:
path = Path('food-101')
path_img = path/'images'

In [8]:
def filelist2df(path):
    df = pd.read_csv(path, delimiter='/', header=None, names=['label', 'name'])
    df['name'] =  df['label'].astype(str) + "/" + df['name'].astype(str) + ".jpg"
    return df

In [9]:
test_path = 'food-101/meta/test.txt'
test_df = filelist2df(test_path)

In [11]:
top_5 = partial(top_k_accuracy, k=5)

In [13]:
bs=16
test_data = (ImageList.from_df(df=test_df, path=path/'images', cols=1)
            .split_none()
            .label_from_df(cols=0)
            .transform(size=512)
            .databunch(bs=bs)
            .normalize(imagenet_stats))


learn = cnn_learner(test_data, arch, metrics=[accuracy, top_5], callback_fns=ShowGraph)
learn.load('food-101-test-e30-512')

Learner(data=ImageDataBunch;

Train: LabelList (25250 items)
x: ImageList
Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512)
y: CategoryList
apple_pie,apple_pie,apple_pie,apple_pie,apple_pie
Path: food-101/images;

Valid: LabelList (0 items)
x: ImageList

y: CategoryList

Path: food-101/images;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (b

Let's load the test set and evaluate using the trained model.

In [14]:
learn.validate(test_data.train_dl)

[1.7312467, tensor(0.8672), tensor(0.9699)]